In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [3]:
data_path = "netflix_titles.csv"
titles = pd.read_csv(data_path)['description']
titles.head()

0    As her father nears the end of his life, filmm...
1    After crossing paths at a party, a Cape Town t...
2    To protect his family from a powerful drug lor...
3    Feuds, flirtations and toilet talk go down amo...
4    In a city of coaching centers known to train I...
Name: description, dtype: object

In [4]:
max_length = max([len(tokenizer.encode(title.strip())) for title in titles])
print("max_length : ",max_length)

max_length :  62


In [5]:
train_titles, test_titles = train_test_split(titles, test_size=10)

In [6]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>',
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [7]:
dataset = NetflixDataset(train_titles, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [8]:
"""
for temp in train_dataset:
    #print(temp[0])
    #print((temp[1]))
    
    #print(len(temp[0]))
    #print(len(temp[1]))
    if len(temp[0])!=len(temp[1]) and len(temp[0])==62:
        print(temp[0])
        print(temp[1])
    
        break

    """

'\nfor temp in train_dataset:\n    #print(temp[0])\n    #print((temp[1]))\n    \n    #print(len(temp[0]))\n    #print(len(temp[1]))\n    if len(temp[0])!=len(temp[1]) and len(temp[0])==62:\n        print(temp[0])\n        print(temp[1])\n    \n        break\n\n    '

In [9]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=100, save_steps=500,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [10]:
next(model.parameters()).is_cuda

True

In [11]:
test_str = "Hello "

generated = tokenizer("<|startoftext|> "+ test_str, return_tensors="pt").input_ids
print(generated.cuda())
print("generated : ",len(generated))
sample_outputs = model.generate(generated.cuda(),no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2)

print(sample_outputs)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257, 18435,   220]], device='cuda:0')
generated :  1


c:\Users\tharh\anaconda3\envs\tharhtet\lib\site-packages\transformers\generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[50257, 18435,   220,   170, 50258, 27332,   226,   229,   171,   225,
           223,   159,   227,   235,  8582,   242,   107, 12520,   234,   230],
        [50257, 18435,   220,   170, 50258, 27332,   226,   229,   171,   225,
           223,   159,   227,   235,  8582,   242,   107, 12520,   234,   231]],
       device='cuda:0')
 Hello � ㅍ🔯 🌈


In [12]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

c:\Users\tharh\anaconda3\envs\tharhtet\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7917
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 7917


  0%|          | 0/7917 [00:00<?, ?it/s]

{'loss': 5.3517, 'learning_rate': 4.943088402681169e-05, 'epoch': 0.01}
{'loss': 2.2332, 'learning_rate': 4.879853294549134e-05, 'epoch': 0.03}
{'loss': 2.1065, 'learning_rate': 4.8166181864170986e-05, 'epoch': 0.04}
{'loss': 2.0882, 'learning_rate': 4.753383078285064e-05, 'epoch': 0.05}


Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


{'loss': 2.0953, 'learning_rate': 4.690147970153029e-05, 'epoch': 0.06}


...

Model weights saved in ./results\checkpoint-7500\pytorch_model.bin


{'loss': 1.9883, 'learning_rate': 2.0045529277855064e-06, 'epoch': 0.96}
{'loss': 1.9983, 'learning_rate': 1.3722018464651576e-06, 'epoch': 0.97}
{'loss': 1.9633, 'learning_rate': 7.398507651448085e-07, 'epoch': 0.99}
{'loss': 1.9329, 'learning_rate': 1.0749968382445935e-07, 'epoch': 1.0}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 464.7978, 'train_samples_per_second': 17.033, 'train_steps_per_second': 17.033, 'train_loss': 2.0455152909834307, 'epoch': 1.0}


TrainOutput(global_step=7917, training_loss=2.0455152909834307, metrics={'train_runtime': 464.7978, 'train_samples_per_second': 17.033, 'train_steps_per_second': 17.033, 'train_loss': 2.0455152909834307, 'epoch': 1.0})

In [15]:
results = []
for title in test_titles: 
    new_titles = {
        'seed': title.split()[0],
        'predictions': []
    }
    generated = tokenizer("<|startoftext|> "+ title.split()[0], return_tensors="pt").input_ids.cuda()
    sample_outputs = model.generate(generated,no_repeat_ngram_size = 1,num_beams=20, num_return_sequences=2)
    
    new_titles['predictions'] = sample_outputs
    results.append(new_titles)
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [14]:
for new_title in results:
    print(f"seed: {new_title['seed']}")
    for i, pred in enumerate(new_title['predictions']):
        print(f"{i+1}: {tokenizer.decode(pred, skip_special_tokens=True)}")

seed: When
1:  When a young man falls in love with his girlfriend, he begins to suspect that she’
2:  When a young man falls in love with his older sister, he's forced to choose between her
seed: Ahead
1:  Ahead of his first birthday, a young man embarks on an epic journey to find the truth
2:  Ahead of his first birthday, a young man embarks on an epic journey to save the world
seed: A
1:  A young man's life is turned upside down when he falls in love with his best friend,
2:  A young man's life is turned upside down when he falls in love with his ex-wife
seed: This
1:  This docuseries takes a look at the rise and fall of Donald Trump's presidential campaign,
2:  This docuseries takes a look at the rise and fall of Donald Trump’s presidential
seed: An
1:  An ex-con returns to the U.S., where he reconnects with his estranged father
2:  An ex-con returns to the U.S., where he meets his estranged wife, who
seed: An
1:  An ex-con returns to his hometown of New York, where he’s haunted by
